# UAV Semantic Segmentation

### Imports

In [1]:
import datetime
import math
import os
import random
import sys

import gdal
import numpy as np
import scipy.misc
import imageio
import tensorflow as tf
from PIL import Image
from skimage import img_as_float
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score
%matplotlib inline
from matplotlib import pyplot as plt

/home/chc/anaconda3/envs/Segmentation/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/chc/anaconda3/envs/Segmentation/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/chc/anaconda3/envs/Segmentation/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/chc/anaconda3/envs/Segmentation

In [2]:
NUM_CLASSES = 6


class BatchColors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

### Define Palette for colors (for labelling)

In [3]:
palette = {0 : (255, 255, 255), # Impervious surfaces (white)
           1 : (0, 0, 255),     # Buildings (blue)
           2 : (0, 255, 255),   # Low vegetation (cyan)
           3 : (0, 255, 0),     # Trees (green)
           4 : (255, 255, 0),   # Cars (yellow)
           5 : (255, 0, 0),     # Clutter (red)
           6 : (0, 0, 0)}       # Undefined (black)


invert_palette = {v: k for k, v in palette.items()}

# Initialization

In [4]:
list_params = ['input_path', 'output_path(for model, images, etc)', 'currentModelPath', 'trainingInstances',
                   'testing_instances', 'learningRate', 'weight_decay', 'batch_size', 'niter', 'reference_crop_size',
                   'reference_stride_crop',
                   'net_type[dilated_icpr_original|dilated_grsl|dilated_icpr_rate6|dilated_icpr_rate6_small|'
                   'dilated_icpr_rate6_densely]',
                   'distribution_type[single_fixed|multi_fixed|uniform|multinomial]', 'probValues',
                   'update_type [acc|loss]', 'process [training|validate_test|generate_final_maps]']

## Initializing individual Parameters


In [5]:
dataset = "postdam"
input_path = "/home/chc/Documents/CVML/dynamic-rs-segmentation/Postdam/"
output_path = "/home/chc/Documents/CVML/dynamic-rs-segmentation/Output/"
former_model_path = "/home/chc/Documents/CVML/dynamic-rs-segmentation/Model/"
trainingInstances = ["2_10","2_11"]
testing_instances = ["2_12"]
lr_initial = 0.01
weight_decay = 0.001
batch_size = 128
niter = 1000
reference_crop_size = 50
reference_stride_crop = 25
net_type = "dilated_icpr_original"
distribution_type = "multinomial"
values = [45,55,65,75,85]
update_type = "acc"
process = "training"
display_step = 50

### Initialize resample_batch

In [6]:
if dataset == 'vaihingen':
    resample_batch = 20
elif dataset == 'postdam':
    resample_batch = 10
else:
    print("Error! No dataset identified: ", dataset)

### Create probability distribution array 

In [7]:
def define_multinomial_probs(values, dif_prob=2):
    interval_size = values[-1] - values[0] + 1

    general_prob = 1.0 / float(interval_size)
    max_prob = general_prob * dif_prob  # for values

    probs = np.full(interval_size, (1.0 - max_prob * len(values)) / float(interval_size - len(values)))
    for i in range(len(values)):
        probs[values[i] - values[0]] = max_prob

    return probs

In [8]:
if distribution_type == 'multi_fixed':
    patch_acc_loss = np.zeros(len(values), dtype=np.float32)
    patch_occur = np.zeros(len(values), dtype=np.int32)
    patch_chosen_values = np.zeros(len(values), dtype=np.int32)

elif distribution_type == 'uniform' or distribution_type == 'multinomial':
    patch_acc_loss = np.zeros(values[-1] - values[0] + 1, dtype=np.float32)
    patch_occur = np.zeros(values[-1] - values[0] + 1, dtype=np.int32)
    patch_chosen_values = np.zeros(values[-1] - values[0] + 1, dtype=np.int32)
    probs = define_multinomial_probs(values)

### Image Loader

In [9]:
def convert_from_color(arr_3d, palette=invert_palette):
    """ RGB-color encoding to grayscale labels """
    arr_2d = np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.uint8)

    for c, i in palette.items():
        m = np.all(arr_3d == np.array(c).reshape(1, 1, 3), axis=2)
        arr_2d[m] = i

    return arr_2d

In [10]:
def load_images(path, instances, process, image_type='vaihingen'):
    images = []
    masks = []

    for f in instances:
        print(BatchColors.OKBLUE + 'Reading instance ' + str(f) + BatchColors.ENDC)
        
        if image_type == 'postdam':
            
            img_ndsm = img_as_float(imageio.imread(path + '1_DSM_normalisation/dsm_potsdam_0' + (
                str(f) if int(f.split("_")[1]) >= 10 else str(f.split("_")[0]) + '_0' + str(
                    f.split("_")[1])) + '_normalized_lastools.jpg'))
            
#             print(type(img_ndsm))
#             plt.imshow(img_ndsm, interpolation='nearest')
#             plt.show()
            
            if len(img_ndsm) != len(img_ndsm[0]):
                new_columns = np.zeros([len(img_ndsm), 1], dtype=type(img_ndsm[0, 0]))
                img_ndsm = np.append(img_ndsm, new_columns, axis=1)
            
            img_ndsm = np.reshape(img_ndsm, (len(img_ndsm), len(img_ndsm[0]), 1))
                        
            ds = gdal.Open(path + '4_Ortho_RGBIR/top_potsdam_' + str(f) + '_RGBIR.tif')
            img_rgb = np.empty([ds.RasterXSize, ds.RasterYSize, ds.RasterCount], dtype=np.float64)
#             print(ds.RasterCount)
#             print(img_rgb.shape)
#             plt.imshow(img_ndsm[:,:,0], interpolation='nearest')
#             plt.show()
            
            for band in range(1, ds.RasterCount + 1):
                img_rgb[:, :, band - 1] = img_as_float(np.array(ds.GetRasterBand(band).ReadAsArray()))

            if process == 'validate_test':
                img_label = imageio.imread(path + 'gts_eroded_encoding/top_potsdam_' + str(f) +
                                              '_label_noBoundary.tif')
            elif process == 'training' or process == 'crf':
                img_label = np.asarray(convert_from_color(imageio.imread(path + '5_Labels_all/top_potsdam_' + str(f) + '_label.tif')))
        

        full_img = np.concatenate((img_rgb, img_ndsm), axis=2)
#         print(full_img.shape)
#         plt.imshow(img_ndsm[:,:,0], interpolation='nearest')
#         plt.show()

        images.append(full_img)
        if process == 'validate_test' or process == 'training' or process == 'crf':
            masks.append(img_label)

    return np.asarray(images), np.asarray(masks)

### Load Training Data

In [11]:
print(BatchColors.WARNING + 'Reading images...' + BatchColors.ENDC)
training_data, training_labels = load_images(input_path, trainingInstances, process, image_type=dataset)
print(training_labels.shape)
print("Training Data Loaded ...")

Reading images...
Reading instance 2_10
Reading instance 2_11
(2, 6000, 6000)
Training Data Loaded ...


### Load Testing Data

In [12]:
testing_data, testing_labels = load_images(input_path, testing_instances, process, image_type=dataset)
print("Testing Data Loaded ...")

Reading instance 2_12
Testing Data Loaded ...


### Create Training Class Distribution

training_class_distribution array has all instances of patches of the classes
\nEg:
training_class_distribution[0] has instances of all 'impervious white surfaces'

In [19]:
# NUM_CLASSES = 256
def create_distributions_over_classes(labels, crop_size, stride_crop):
    classes = [[[] for i in range(0)] for i in range(NUM_CLASSES)]
#     print(classes)
#     print(len(classes))
#     print(len(labels))
#     print("------------------")
    for k in range(len(labels)):
        w, h = labels[k].shape

        for i in range(0, w, stride_crop):
            for j in range(0, h, stride_crop):
                cur_map = k
                cur_x = i
                cur_y = j
                patch_class = labels[cur_map][cur_x:cur_x + crop_size, cur_y:cur_y + crop_size]
#                 plt.imshow(patch_class, interpolation='nearest')
#                 plt.show()
#                 print(patch_class)

                if len(patch_class) != crop_size and len(patch_class[0]) != crop_size:
                    cur_x = cur_x - (crop_size - len(patch_class))
                    cur_y = cur_y - (crop_size - len(patch_class[0]))
                    patch_class = labels[cur_map][cur_x:cur_x + crop_size, cur_y:cur_y + crop_size]
                elif len(patch_class) != crop_size:
                    cur_x = cur_x - (crop_size - len(patch_class))
                    patch_class = labels[cur_map][cur_x:cur_x + crop_size, cur_y:cur_y + crop_size]
                elif len(patch_class[0]) != crop_size:
                    cur_y = cur_y - (crop_size - len(patch_class[0]))
                    patch_class = labels[cur_map][cur_x:cur_x + crop_size, cur_y:cur_y + crop_size]

                if patch_class.shape == (crop_size, crop_size):
#                     print(patch_class.shape)
#                     print(patch_class.astype(int).flatten())
                    count = np.bincount(patch_class.astype(int).flatten())
#                     print(count)
#                     print(len(count))
#                     print(int(np.argmax(count)))
                    classes[int(np.argmax(count))].append((cur_map, cur_x, cur_y))
#                     print(classes)
                else:
                    print(BatchColors.FAIL + "Error create_distributions_over_classes: Current patch size is " + str(
                        len(patch_class)) + "x" + str(len(patch_class[0])) + BatchColors.ENDC)
                    return

    for i in range(len(classes)):
        print(BatchColors.OKBLUE + 'Class ' + str(i + 1) + ' has length ' + str(len(classes[i])) + BatchColors.ENDC)

    return classes

In [25]:
print(BatchColors.WARNING + 'Creating TRAINING class distribution...' + BatchColors.ENDC)
training_class_distribution = create_distributions_over_classes(training_labels, crop_size=reference_crop_size,
                                                                        stride_crop=reference_stride_crop)
print(training_class_distribution[0])

Creating TRAINING class distribution...
Class 1 has length 25281
Class 2 has length 14755
Class 3 has length 45616
Class 4 has length 16595
Class 5 has length 732
Class 6 has length 12221
[(0, 0, 275), (0, 0, 300), (0, 0, 325), (0, 0, 350), (0, 0, 375), (0, 0, 450), (0, 0, 1225), (0, 0, 1575), (0, 0, 1600), (0, 0, 2000), (0, 0, 2025), (0, 0, 2050), (0, 0, 2075), (0, 0, 2100), (0, 0, 2125), (0, 0, 2150), (0, 0, 2175), (0, 0, 2200), (0, 0, 2225), (0, 0, 2250), (0, 0, 2275), (0, 0, 2300), (0, 0, 2325), (0, 0, 2350), (0, 0, 2375), (0, 0, 2400), (0, 0, 2425), (0, 0, 2450), (0, 0, 2475), (0, 0, 2500), (0, 0, 2525), (0, 0, 2550), (0, 0, 2575), (0, 0, 2600), (0, 0, 2625), (0, 0, 2650), (0, 0, 2675), (0, 0, 2700), (0, 0, 2725), (0, 0, 2750), (0, 0, 2775), (0, 0, 2800), (0, 0, 2825), (0, 0, 2850), (0, 0, 2875), (0, 0, 2900), (0, 0, 5275), (0, 0, 5300), (0, 0, 5500), (0, 0, 5525), (0, 0, 5550), (0, 0, 5575), (0, 0, 5675), (0, 0, 5700), (0, 0, 5725), (0, 0, 5750), (0, 0, 5900), (0, 0, 5925), (0, 0

### Create Rotation Distribution

In [34]:
def create_rotation_distribution(training_class_distribution):
    rotation = [[[] for i in range(0)] for i in range(NUM_CLASSES)]
    print(rotation)

    for i in range(len(training_class_distribution)):
        # print len(training_class_distribution[i])
        rotation[i] = np.random.randint(0, 360, size=len(training_class_distribution[i]))

    for i in range(len(training_class_distribution)):
        print(BatchColors.OKBLUE + 'Class ' + str(i + 1) + ' has length ' + str(
            len(training_class_distribution[i])) + ' and rotation length ' + str(len(rotation[i])) + BatchColors.ENDC)
    return rotation

In [36]:
if os.path.isfile(os.getcwd() + '/dataset_' + dataset + '_crop_' + str(reference_crop_size) + '_stride_' + str(
            reference_stride_crop) + '_rotation.npy'):
        training_rotation_distribution = np.load(
            os.getcwd() + '/dataset_' + dataset + '_crop_' + str(reference_crop_size) + '_stride_' + str(
                reference_stride_crop) + '_rotation.npy')
        print(BatchColors.OKGREEN + 'Loaded training instance rotations' + BatchColors.ENDC)
else:
    training_rotation_distribution = create_rotation_distribution(training_class_distribution)
    np.save(os.getcwd() + '/dataset_' + dataset + '_crop_' + str(reference_crop_size) + '_stride_' + str(
        reference_stride_crop) + '_rotation.npy', training_rotation_distribution)
    print(BatchColors.OKGREEN + 'Created training instance rotations' + BatchColors.ENDC)
    print(len(training_rotation_distribution[0]))
    print(training_rotation_distribution[0])    

[[], [], [], [], [], []]
Class 1 has length 25281 and rotation length 25281
Class 2 has length 14755 and rotation length 14755
Class 3 has length 45616 and rotation length 45616
Class 4 has length 16595 and rotation length 16595
Class 5 has length 732 and rotation length 732
Class 6 has length 12221 and rotation length 12221
Created training instance rotations
25281
[ 57 104 306 ... 303 293  16]


### Create Mean and Std 

In [75]:
def compute_image_mean(data):
    mean_full = np.mean(np.mean(np.mean(data, axis=0), axis=0), axis=0)
    std_full = np.std(data, axis=0, ddof=1)[0, 0, :]

    return mean_full, std_full

In [82]:
def dynamically_calculate_mean_and_std(data, indexes, crop_size):
    total = indexes[0] + indexes[1] + indexes[2] + indexes[3] + indexes[4] + indexes[5]

    mean_full = []
    std_full = []

    all_patches = []
    # count = 0

    ''' For every patch in the actual training image, do the following : '''
    for i in range(len(total)):
        cur_map = total[i][0]
        cur_x = total[i][1]
        cur_y = total[i][2]

        patches = data[cur_map][cur_x:cur_x + crop_size, cur_y:cur_y + crop_size, :]
#         print(type(patches))
#         plt.imshow(patches[:,:,1], interpolation='nearest')
#         plt.show()
        if len(patches[0]) != crop_size or len(patches[1]) != crop_size:
            print(BatchColors.FAIL + "Error! Current patch size: " + str(len(patches)) + "x" + \
                  str(len(patches[0])) + BatchColors.ENDC)
            return

        all_patches.append(patches)

        if i > 0 and i % 5000 == 0:
            mean, std = compute_image_mean(np.asarray(all_patches))
            mean_full.append(mean)
            std_full.append(std)
            all_patches = []

    print(len(all_patches)*5000)
    # remaining images
    mean, std = compute_image_mean(np.asarray(all_patches))
    mean_full.append(mean)
    std_full.append(std)

    # print 'count', count
    return np.mean(mean_full, axis=0), np.mean(std_full, axis=0)

In [83]:
if os.path.isfile(os.getcwd() + '/dataset_' + dataset + '_crop_' + str(reference_crop_size) + '_stride_' + str(
            reference_stride_crop) + '_mean.npy'):
        mean_full = np.load(
            os.getcwd() + '/dataset_' + dataset + '_crop_' + str(reference_crop_size) + '_stride_' + str(
                reference_stride_crop) + '_mean.npy')
        std_full = np.load(os.getcwd() + '/dataset_' + dataset + '_crop_' + str(reference_crop_size) + '_stride_' + str(
            reference_stride_crop) + '_std.npy')
        print(BatchColors.OKGREEN + 'Loaded Mean/Std from training instances' + BatchColors.ENDC)
else:
    mean_full, std_full = dynamically_calculate_mean_and_std(training_data, training_class_distribution,
                                                             crop_size=25)
    np.save(os.getcwd() + '/dataset_' + dataset + '_crop_' + str(reference_crop_size) + '_stride_' + str(
        reference_stride_crop) + '_mean.npy', mean_full)
    np.save(os.getcwd() + '/dataset_' + dataset + '_crop_' + str(reference_crop_size) + '_stride_' + str(
        reference_stride_crop) + '_std.npy', std_full)
    print(BatchColors.OKGREEN + 'Created Mean/Std from training instances' + BatchColors.ENDC)

Loaded Mean/Std from training instances
